In [128]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [129]:
data=pd.read_csv('./data.csv')

In [130]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [131]:
data.groupby('Category').describe().T

Category                           ham  \
Message count                     4825   
        unique                    4516   
        top     Sorry, I'll call later   
        freq                        30   

Category                                                     spam  
Message count                                                 747  
        unique                                                641  
        top     Please call our customer service representativ...  
        freq                                                    4

In [132]:
ham_data = data[data.Category == 'ham']
spam_data = data[data.Category == 'spam']
print(spam_data.shape)
print(ham_data.shape)
print(data.shape)

(747, 2)
(4825, 2)
(5572, 2)


In [135]:
ham_data=ham_data.sample(n=len(spam_data),random_state=42)
print(ham_data.shape,spam_data.shape)

(747, 2) (747, 2)


In [166]:
balanced_data=ham_data.append(spam_data).reset_index(drop=True)
print(balanced_data.shape)

(1494, 2)


C:\Users\subek\AppData\Local\Temp\ipykernel_10008\230600876.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_data=ham_data.append(spam_data).reset_index(drop=True)


In [167]:
balanced_data.head()

,Category,Message
0,ham,Is that seriously how you spell his name?
1,ham,"Hey elaine, is today's meeting still on?"
2,ham,Watching telugu movie..wat abt u?
3,ham,"Short But Cute: ""Be a good person, but dont tr..."
4,ham,I will vote for wherever my heart guides me


In [172]:
balanced_data['label']=balanced_data['Category'].map({'ham':0,'spam':1})

In [173]:
balanced_data.head()

,Category,Message,label
0,ham,Is that seriously how you spell his name?,0
1,ham,"Hey elaine, is today's meeting still on?",0
2,ham,Watching telugu movie..wat abt u?,0
3,ham,"Short But Cute: ""Be a good person, but dont tr...",0
4,ham,I will vote for wherever my heart guides me,0


In [181]:
train_data, test_data, train_labels, test_labels =train_test_split(balanced_data['Message'],balanced_data['label'],test_size=0.2,random_state=434)

In [182]:
vocab_size=500
oov_tok='<OOV>'
max_len=50

In [183]:
token=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
token.fit_on_texts(train_data)

In [184]:
word_index=token.word_index
word_index

{'<OOV>': 1,
 'to': 2,
 'you': 3,
 'a': 4,
 'i': 5,
 'call': 6,
 'the': 7,
 'your': 8,
 'u': 9,
 'for': 10,
 '2': 11,
 'is': 12,
 'now': 13,
 'and': 14,
 'free': 15,
 'or': 16,
 'in': 17,
 'on': 18,
 'have': 19,
 'ur': 20,
 'of': 21,
 'txt': 22,
 '4': 23,
 'with': 24,
 'from': 25,
 'me': 26,
 'are': 27,
 'text': 28,
 'this': 29,
 'get': 30,
 'stop': 31,
 'mobile': 32,
 'my': 33,
 'just': 34,
 'no': 35,
 'will': 36,
 'reply': 37,
 'be': 38,
 'claim': 39,
 'it': 40,
 'only': 41,
 'www': 42,
 'that': 43,
 'so': 44,
 'do': 45,
 'new': 46,
 'not': 47,
 'send': 48,
 'we': 49,
 'out': 50,
 'our': 51,
 'prize': 52,
 'cash': 53,
 'can': 54,
 'at': 55,
 "i'm": 56,
 'but': 57,
 'if': 58,
 '150p': 59,
 'uk': 60,
 'all': 61,
 'who': 62,
 'nokia': 63,
 '1': 64,
 'please': 65,
 'win': 66,
 't': 67,
 'tone': 68,
 'won': 69,
 'by': 70,
 'go': 71,
 'week': 72,
 'been': 73,
 'com': 74,
 'what': 75,
 'msg': 76,
 '50': 77,
 'want': 78,
 'phone': 79,
 'day': 80,
 'service': 81,
 'urgent': 82,
 'when': 83,
 